# Analysis

In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go  # Import graph_objects as go
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Initialize the Dash app
app = dash.Dash(__name__, title="Google Ads Performance Dashboard")
server = app.server  # For deployment purposes

# Sample Data Creation (Replace this with actual Google Ads API data fetching)
def create_mock_data():
    np.random.seed(42)  # For reproducibility
    date_range = pd.date_range(start="2024-01-01", end="2024-12-31")
    campaigns = ['Campaign A', 'Campaign B', 'Campaign C']
    regions = ['North America', 'Europe', 'Asia', 'South America']
    devices = ['Desktop', 'Mobile', 'Tablet']

    data = []
    for campaign in campaigns:
        for date in date_range:
            impressions = np.random.randint(1000, 10000)
            clicks = np.random.randint(50, 1000)
            ctr = clicks / impressions * 100
            cpc = np.round(np.random.uniform(0.5, 5.0), 2)
            conversions = np.random.randint(10, 200)
            conversion_rate = conversions / clicks * 100 if clicks > 0 else 0
            cpa = np.round(np.random.uniform(10, 100), 2) if conversions > 0 else 0
            roas = np.round(np.random.uniform(1, 10), 2)
            quality_score = np.random.randint(1, 10)
            ad_position = np.round(np.random.uniform(1, 10), 2)
            budget_spent = np.round(impressions * cpc / 1000, 2)
            revenue = roas * budget_spent
            region = np.random.choice(regions)
            device = np.random.choice(devices)

            data.append({
                'Date': date,
                'Campaign': campaign,
                'Impressions': impressions,
                'Clicks': clicks,
                'CTR': ctr,
                'CPC': cpc,
                'Conversions': conversions,
                'Conversion Rate': conversion_rate,
                'CPA': cpa,
                'ROAS': roas,
                'Quality Score': quality_score,
                'Ad Position': ad_position,
                'Budget Spent': budget_spent,
                'Revenue': revenue,
                'Region': region,
                'Device': device
            })
    df = pd.DataFrame(data)
    return df

# Load Data
df = create_mock_data()

# Precompute some metrics
df['Date'] = pd.to_datetime(df['Date'])
max_roas_threshold = df['ROAS'].max() + 2  # For gauge

# App Layout
app.layout = html.Div([
    # Header
    html.Div([
        html.H1("Google Ads Performance Dashboard", style={'textAlign': 'center'}),
        html.P("Analyze your Google Ads performance with comprehensive metrics.", style={'textAlign': 'center'})
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa'}),

    # Filters
    html.Div([
        html.Div([
            html.Label("Select Date Range:"),
            dcc.DatePickerRange(
                id='date-picker',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                start_date=df['Date'].min(),
                end_date=df['Date'].max()
            )
        ], style={'display': 'inline-block', 'marginRight': '20px'}),

        html.Div([
            html.Label("Select Campaign:"),
            dcc.Dropdown(
                id='campaign-dropdown',
                options=[{'label': camp, 'value': camp} for camp in df['Campaign'].unique()],
                value=df['Campaign'].unique().tolist(),
                multi=True
            )
        ], style={'display': 'inline-block', 'width': '200px', 'marginRight': '20px'}),

        html.Div([
            html.Label("Select Region:"),
            dcc.Dropdown(
                id='region-dropdown',
                options=[{'label': reg, 'value': reg} for reg in df['Region'].unique()],
                value=df['Region'].unique().tolist(),
                multi=True
            )
        ], style={'display': 'inline-block', 'width': '200px', 'marginRight': '20px'}),

        html.Div([
            html.Label("Select Device:"),
            dcc.Dropdown(
                id='device-dropdown',
                options=[{'label': dev, 'value': dev} for dev in df['Device'].unique()],
                value=df['Device'].unique().tolist(),
                multi=True
            )
        ], style={'display': 'inline-block', 'width': '200px'})
    ], style={'padding': '20px', 'backgroundColor': '#e9ecef'}),

    # Main Content
    html.Div([
        # Row 1: Impressions vs Clicks and CTR & CPC
        html.Div([
            html.Div([
                dcc.Graph(id='impressions-clicks')
            ], className='six columns'),

            html.Div([
                dcc.Graph(id='ctr-cpc')
            ], className='six columns'),
        ], className='row'),

        # Row 2: Conversion Rate & CPA and ROAS
        html.Div([
            html.Div([
                dcc.Graph(id='conversion-cpa')
            ], className='six columns'),

            html.Div([
                dcc.Graph(id='roas-gauge')
            ], className='six columns'),
        ], className='row'),

        # Row 3: Quality Score and Ad Position
        html.Div([
            html.Div([
                dcc.Graph(id='quality-score')
            ], className='six columns'),

            html.Div([
                dcc.Graph(id='ad-position')
            ], className='six columns'),
        ], className='row'),

        # Row 4: Budget Utilization and Geographic Performance
        html.Div([
            html.Div([
                dcc.Graph(id='budget-utilization')
            ], className='six columns'),

            html.Div([
                dcc.Graph(id='geographic-performance')
            ], className='six columns'),
        ], className='row'),

        # Row 5: Device Performance Breakdown
        html.Div([
            html.Div([
                dcc.Graph(id='device-performance')
            ], className='twelve columns'),
        ], className='row'),

    ], style={'padding': '20px'}),

    # Footer
    html.Div([
        html.P("© 2024 Your Company Name. All rights reserved.", style={'textAlign': 'center'})
    ], style={'padding': '10px', 'backgroundColor': '#f8f9fa'})
], style={'fontFamily': 'Arial, sans-serif'})

# Callback to update all graphs based on filters
@app.callback(
    [
        Output('impressions-clicks', 'figure'),
        Output('ctr-cpc', 'figure'),
        Output('conversion-cpa', 'figure'),
        Output('roas-gauge', 'figure'),
        Output('quality-score', 'figure'),
        Output('ad-position', 'figure'),
        Output('budget-utilization', 'figure'),
        Output('geographic-performance', 'figure'),
        Output('device-performance', 'figure'),
    ],
    [
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date'),
        Input('campaign-dropdown', 'value'),
        Input('region-dropdown', 'value'),
        Input('device-dropdown', 'value')
    ]
)
def update_graphs(start_date, end_date, selected_campaigns, selected_regions, selected_devices):
    # Filter data based on selections
    filtered_df = df[
        (df['Date'] >= pd.to_datetime(start_date)) &
        (df['Date'] <= pd.to_datetime(end_date)) &
        (df['Campaign'].isin(selected_campaigns)) &
        (df['Region'].isin(selected_regions)) &
        (df['Device'].isin(selected_devices))
    ]

    # Check if filtered_df is empty to prevent errors
    if filtered_df.empty:
        # Return empty figures or figures with no data
        empty_fig = go.Figure()
        empty_fig.update_layout(
            title="No data available for the selected filters."
        )
        return [empty_fig]*9

    # Impressions vs Clicks
    impressions_clicks_data = filtered_df.groupby('Date').agg({'Impressions': 'sum', 'Clicks': 'sum'}).reset_index()
    impressions_clicks = px.bar(
        impressions_clicks_data,
        x='Date',
        y='Impressions',
        labels={'Impressions': 'Impressions', 'Date': 'Date'},
        title='Impressions vs Clicks Over Time'
    )
    # Add Clicks as a line trace
    clicks_line = px.line(
        impressions_clicks_data,
        x='Date',
        y='Clicks',
        labels={'Clicks': 'Clicks'},
        title=''
    )
    for trace in clicks_line.data:
        impressions_clicks.add_trace(trace)
    impressions_clicks.update_layout(
        yaxis_title='Count',
        legend_title='Metrics'
    )

    # CTR and CPC
    ctr_cpc = px.scatter(
        filtered_df,
        x='CTR',
        y='CPC',
        color='Campaign',
        size='Clicks',
        hover_data=['Campaign', 'Date'],
        title='CTR vs CPC by Campaign',
        labels={'CTR': 'Click-Through Rate (%)', 'CPC': 'Cost Per Click ($)'}
    )

    # Conversion Rate and CPA
    conversion_cpa = px.scatter(
        filtered_df,
        x='Conversion Rate',
        y='CPA',
        color='Campaign',
        size='Conversions',
        hover_data=['Campaign', 'Date'],
        title='Conversion Rate vs CPA by Campaign',
        labels={'Conversion Rate': 'Conversion Rate (%)', 'CPA': 'Cost Per Acquisition ($)'}
    )

    # ROAS Gauge (Average ROAS)
    avg_roas = filtered_df['ROAS'].mean()
    roas_gauge = go.Figure(go.Indicator(
        mode="gauge+number",
        value=avg_roas,
        title={'text': "Average ROAS"},
        gauge={
            'axis': {'range': [None, max_roas_threshold]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, max_roas_threshold * 0.5], 'color': "#ff6666"},
                {'range': [max_roas_threshold * 0.5, max_roas_threshold * 0.8], 'color': "#ffcc66"},
                {'range': [max_roas_threshold * 0.8, max_roas_threshold], 'color': "#66ff66"}
            ]
        }
    ))
    roas_gauge.update_layout(height=300)

    # Quality Score Distribution
    quality_score = px.histogram(
        filtered_df,
        x='Quality Score',
        nbins=10,
        title='Quality Score Distribution',
        labels={'Quality Score': 'Quality Score', 'count': 'Number of Ads'},
        color='Campaign',
        barmode='overlay',
        opacity=0.7
    )

    # Ad Position Trends
    ad_position_data = filtered_df.groupby('Date').agg({'Ad Position': 'mean'}).reset_index()
    ad_position = px.line(
        ad_position_data,
        x='Date',
        y='Ad Position',
        title='Average Ad Position Over Time',
        labels={'Ad Position': 'Average Ad Position', 'Date': 'Date'},
    )
    ad_position.update_yaxes(autorange="reversed")  # Lower ad position number means higher placement

    # Budget Utilization
    budget_utilization_data = filtered_df.groupby('Date').agg({'Budget Spent': 'sum'}).reset_index()
    budget_utilization = px.bar(
        budget_utilization_data,
        x='Date',
        y='Budget Spent',
        title='Budget Spent Over Time',
        labels={'Budget Spent': 'Budget Spent ($)', 'Date': 'Date'},
    )
    # Assuming a fixed monthly budget for demonstration
    monthly_budget = 5000  # Replace with actual budget data
    budget_utilization.add_shape(
        type="line",
        x0=budget_utilization_data['Date'].min(),
        y0=monthly_budget,
        x1=budget_utilization_data['Date'].max(),
        y1=monthly_budget,
        line=dict(color="Red", dash="dash"),
        name="Monthly Budget"
    )
    budget_utilization.add_annotation(
        x=budget_utilization_data['Date'].min(),
        y=monthly_budget,
        text="Monthly Budget",
        showarrow=True,
        arrowhead=1
    )

    # Geographic Performance Map
    geo_data = filtered_df.groupby('Region').agg({
        'Impressions': 'sum',
        'Clicks': 'sum',
        'Conversions': 'sum',
        'Revenue': 'sum',
        'Budget Spent': 'sum'  # Added 'Budget Spent' to aggregation
    }).reset_index()
    # Calculate ROAS accurately
    # Avoid division by zero by replacing zero Budget Spent with NaN
    geo_data['ROAS'] = geo_data.apply(
        lambda row: row['Revenue'] / row['Budget Spent'] if row['Budget Spent'] > 0 else np.nan,
        axis=1
    )

    # Handle possible NaN ROAS values by setting them to zero or another appropriate value
    geo_data['ROAS'].fillna(0, inplace=True)

    # Ensure that 'Region' corresponds to recognized country names for Plotly's choropleth
    # If 'Region' refers to continents or custom regions, consider using a different approach or mapping
    geographic_performance = px.choropleth(
        geo_data,
        locations='Region',
        locationmode='country names',  # Ensure 'Region' contains valid country names
        color='ROAS',
        hover_name='Region',
        hover_data={
            'Impressions': True,
            'Clicks': True,
            'Conversions': True,
            'Revenue': True,
            'ROAS': ':.2f'
        },
        color_continuous_scale='Blues',
        title='ROAS by Region'
    )

    # Device Performance Breakdown
    device_performance = px.pie(
        filtered_df,
        names='Device',
        values='Clicks',
        title='Clicks Distribution by Device',
        hole=0.3
    )

    return (
        impressions_clicks,
        ctr_cpc,
        conversion_cpa,
        roas_gauge,
        quality_score,
        ad_position,
        budget_utilization,
        geographic_performance,
        device_performance
    )

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


NameError: name 'plotly' is not defined